In [ ]:
import os
import cv2
import tqdm
import easyocr
from yt_dlp import YoutubeDL

In [ ]:

def extract_text_from_youtube(video_url, output_folder, lang='en'):
    """
    Downloads a YouTube video (360p), extracts text from frames, crops detected text, and saves it.
    :param video_url: URL of the YouTube video.
    :param output_folder: Folder to save cropped images and text file.
    :param lang: Language code for OCR.
    """
    reader = easyocr.Reader([lang])
    os.makedirs(output_folder, exist_ok=True)
    label_file = os.path.join(output_folder, "labels.txt")
    
    with open(label_file, 'w', encoding='utf-8') as f:
        f.write("")

    ydl_opts = {}
    ydl = YoutubeDL(ydl_opts)
    info_dict = ydl.extract_info(video_url, download=False)
    formats = info_dict.get('formats', None)

    video_url = next((f.get('url') for f in formats if f.get('format_note') == '360p'), None)
    if not video_url:
        print("No 360p video format found.")
        return

    cap = cv2.VideoCapture(video_url)
    if not cap.isOpened():
        print("Could not open video stream.")
        return

    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    shot_index = 0
    pbar = tqdm.tqdm(total=total_frames // 70, desc="Processing frames")

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        result = reader.readtext(frame, detail=1)

        for i, (bbox, text, _) in enumerate(result):
            x_min, y_min = map(int, bbox[0])
            x_max, y_max = map(int, bbox[2])
            cropped_img = frame[y_min:y_max, x_min:x_max]
            
            if cropped_img.shape[0] > cropped_img.shape[1]:  
                continue
            
            cropped_text_result = reader.readtext(cropped_img, detail=0)
            cropped_text = " ".join(cropped_text_result) if cropped_text_result else ""
            
            if cropped_text:
                shot_name = f"shot_{shot_index}.png"
                cropped_path = os.path.join(output_folder, shot_name)
                cv2.imwrite(cropped_path, cropped_img)
                
                with open(label_file, 'a', encoding='utf-8') as f:
                    f.write(f"{shot_name},{cropped_text}\n")
                
                shot_index += 1

        cap.set(cv2.CAP_PROP_POS_FRAMES, cap.get(cv2.CAP_PROP_POS_FRAMES) + 70)
        pbar.update(1)
        
    cap.release()
    pbar.close()
    print(f"Saved cropped images in: {output_folder}")
    print(f"Extracted text saved in: {label_file}")


In [ ]:
video_url = "https://www.youtube.com/watch?v=xo_KMaHo4_4&t=4s"  # Change this to your YouTube video link
output_folder = ""
extract_text_from_youtube(video_url, output_folder)

In [ ]:
def find_missing_files(file1, file2):
    """
    Compares two label text files and finds missing file entries. Some images has to be deleted manually.
    :param file1: Path to the first label file.
    :param file2: Path to the second label file.
    """
    def get_filenames(label_file):
        filenames = set()
        with open(label_file, 'r', encoding='utf-8') as f:
            for line in f:
                parts = line.strip().split(',')
                if parts:
                    filenames.add(parts[0])  # First element is the filename
        return filenames
    
    files1 = get_filenames(file1)
    files2 = get_filenames(file2)
    
    missing_files = files1 - files2  # Files present in file1 but missing in file2
    
    print("Missing files:")
    for missing in sorted(missing_files):
        print(missing)
    
    return missing_files

# Case with watermarks

In [ ]:
def crop_image(input_image_path, output_image_path, crop_percent=8):
    """
    Crops the top portion of an image by a given percentage and saves the result.
    :param input_image_path: Path to the input image file.
    :param output_image_path: Path to save the cropped output image.
    :param crop_percent: Percentage of the image height to crop from the top. Defaults to 8.
    """
    image = cv2.imread(input_image_path)
    if image is None:
        print(f"Cannot open file: {input_image_path}")
        return

    height, width, _ = image.shape

    crop_height = int(height * (crop_percent / 100))

    cropped_image = image[crop_height:, :]

    cv2.imwrite(output_image_path, cropped_image)
